# Smart Decisions with Hotel Booking Analytics

## Project Brief
This notebook documents the process of analyzing a real-world hotel booking dataset. The goal is to perform extensive data cleaning, conduct Exploratory Data Analysis (EDA), and uncover meaningful insights and patterns to inform business recommendations.

## Deliverables
- **Jupyter Notebook (.ipynb):** This file, containing full cleaning and EDA.
- **Cleaned CSV:** `hotel_bookings_Cleaned.csv`.
- **Excel Dashboard:** `hotel_bookings_Dashboard.xlsx` (with data ready for PivotTables).
- **Documentation:** `README.md` and Presentation Slides.

## Data Source
The data was loaded from the provided `hotel_bookings.csv` file. The column names were determined through external research as the file lacked a header.


## 0. Setup and Initial Data Loading

In [ ]:

import pandas as pd
import numpy as np

# 1. Define the column names as identified from external research
column_names = [
    'hotel', 'is_canceled', 'lead_time', 'arrival_date_year', 'arrival_date_month',
    'arrival_date_week_number', 'arrival_date_day_of_month', 'stays_in_weekend_nights',
    'stays_in_week_nights', 'adults', 'children', 'babies', 'meal', 'country',
    'market_segment', 'distribution_channel', 'is_repeated_guest', 'previous_cancellations',
    'previous_bookings_not_canceled', 'reserved_room_type', 'assigned_room_type',
    'booking_changes', 'deposit_type', 'agent', 'company', 'days_in_waiting_list',
    'customer_type', 'adr', 'required_car_parking_spaces', 'total_of_special_requests',
    'reservation_status', 'reservation_status_date'
]

# 2. Load the dataset
# The provided CSV does not have a header, so we use the defined column names.
# We also treat 'NULL' strings as NaN based on the initial file inspection.
df = pd.read_csv(
    '/home/ubuntu/upload/hotel_bookings.csv',
    header=None,
    names=column_names,
    na_values=['NULL']
)

print("Initial Data Shape:", df.shape)
print("\nInitial Data Info:")
df.info()
print("\nMissing Values Count:")
print(df.isnull().sum())


## 1. Data Cleaning and Preprocessing

The initial assessment revealed a spurious header row (row 0) and several columns with missing values (`country`, `agent`, `company`, `children`) and incorrect data types (all columns were initially loaded as `object` due to the missing header and mixed types).

### Cleaning Steps:
1.  **Drop Header Row:** Remove the first row which contains the column names repeated as data.
2.  **Type Conversion:** Convert all numeric columns to their correct integer or float types, coercing errors to NaN for further handling.
3.  **Missing Value Imputation:**
    *   `country`: Imputed with 'Unknown'.
    *   `agent` and `company`: Imputed with 0 (representing 'no agent' or 'no company').
    *   `children`: Imputed with 0 (only 4 missing values).
    *   `adr` (Average Daily Rate): Imputed with the median.
4.  **Invalid Data Removal:** Remove records where `adults`, `children`, and `babies` are all zero (invalid bookings).
5.  **Feature Engineering:** Create a single `arrival_date` column from the separate year, month, and day columns.


In [ ]:

# 3. Drop the incorrect header row (Row 0)
df = df.iloc[1:].copy()

# 4. Convert columns to numeric types
numeric_cols = [
    'is_canceled', 'lead_time', 'arrival_date_year', 'arrival_date_week_number',
    'arrival_date_day_of_month', 'stays_in_weekend_nights', 'stays_in_week_nights',
    'adults', 'children', 'babies', 'is_repeated_guest', 'previous_cancellations',
    'previous_bookings_not_canceled', 'booking_changes', 'agent', 'company',
    'days_in_waiting_list', 'adr', 'required_car_parking_spaces', 'total_of_special_requests'
]

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# 5. Handle Missing Values (NaN)
df['country'].fillna('Unknown', inplace=True)
df['agent'].fillna(0, inplace=True)
df['company'].fillna(0, inplace=True)
df['children'].fillna(0, inplace=True)
df['adr'].fillna(df['adr'].median(), inplace=True)

# 6. Handle Invalid Data
zero_guests = (df['adults'] == 0) & (df['children'] == 0) & (df['babies'] == 0)
df.drop(df[zero_guests].index, inplace=True)

# 7. Final Data Type Correction (for integer columns)
integer_cols = [
    'is_canceled', 'lead_time', 'arrival_date_year', 'arrival_date_week_number',
    'arrival_date_day_of_month', 'stays_in_weekend_nights', 'stays_in_week_nights',
    'adults', 'children', 'babies', 'is_repeated_guest', 'previous_cancellations',
    'previous_bookings_not_canceled', 'booking_changes', 'agent', 'company',
    'days_in_waiting_list', 'required_car_parking_spaces', 'total_of_special_requests'
]

for col in integer_cols:
    df[col] = df[col].astype(int)

# 8. Create a full arrival date column
df['arrival_date'] = pd.to_datetime(
    df['arrival_date_year'].astype(str) + '-' + df['arrival_date_month'] + '-' + df['arrival_date_day_of_month'].astype(str),
    format='%Y-%B-%d',
    errors='coerce'
)

print("Cleaned Data Shape:", df.shape)
print("\nFinal Missing Values Count:")
print(df.isnull().sum())


## 2. Exploratory Data Analysis (EDA)

With the data cleaned, we proceed to analyze key trends and patterns.

### Key Metrics


In [ ]:

total_bookings = len(df)
cancellation_rate = df['is_canceled'].mean() * 100
city_hotel_bookings = df[df['hotel'] == 'City Hotel'].shape[0]
resort_hotel_bookings = df[df['hotel'] == 'Resort Hotel'].shape[0]
top_country = df['country'].value_counts().index[0]

print(f"Total Bookings Analyzed: {total_bookings:,}")
print(f"Overall Cancellation Rate: {cancellation_rate:.2f}%")
print(f"City Hotel Bookings: {city_hotel_bookings:,}")
print(f"Resort Hotel Bookings: {resort_hotel_bookings:,}")
print(f"Top Origin Country: {top_country}")


### Visualizations

We will now generate key visualizations to illustrate the findings. The plots are saved to the `eda_visualizations` directory.


In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns
import os

# Create a directory for saving visualizations
output_dir = 'eda_visualizations'
os.makedirs(output_dir, exist_ok=True)

# 1. Cancellation Rate by Hotel Type
cancellation_data = df.groupby('hotel')['is_canceled'].agg(['mean']).reset_index()
cancellation_data['mean'] = cancellation_data['mean'] * 100

plt.figure(figsize=(8, 6))
sns.barplot(x='hotel', y='mean', data=cancellation_data, palette='viridis')
plt.title('Cancellation Rate by Hotel Type', fontsize=16)
plt.xlabel('Hotel Type', fontsize=12)
plt.ylabel('Cancellation Rate (%)', fontsize=12)
plt.ylim(0, 50)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()
plt.savefig(os.path.join(output_dir, 'cancellation_rate_by_hotel.png'))
plt.close()

# 2. Bookings Over Time
df['arrival_date'] = pd.to_datetime(df['arrival_date'])
monthly_bookings = df.groupby(df['arrival_date'].dt.to_period('M')).size().reset_index(name='Total Bookings')
monthly_bookings['arrival_date'] = monthly_bookings['arrival_date'].astype(str)

plt.figure(figsize=(14, 6))
sns.lineplot(x='arrival_date', y='Total Bookings', data=monthly_bookings)
plt.title('Total Bookings Over Time (Monthly)', fontsize=16)
plt.xlabel('Arrival Month', fontsize=12)
plt.ylabel('Total Bookings', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()
plt.savefig(os.path.join(output_dir, 'bookings_over_time.png'))
plt.close()

# 3. Top 10 Countries by Booking Volume
top_n = 10
country_counts = df['country'].value_counts().nlargest(top_n).reset_index()
country_counts.columns = ['Country', 'Bookings']

plt.figure(figsize=(10, 6))
sns.barplot(x='Country', y='Bookings', data=country_counts, palette='Spectral')
plt.title(f'Top {top_n} Countries by Booking Volume', fontsize=16)
plt.xlabel('Country', fontsize=12)
plt.ylabel('Number of Bookings', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()
plt.savefig(os.path.join(output_dir, 'top_countries.png'))
plt.close()

# 4. ADR Distribution by Hotel Type and Cancellation Status
plt.figure(figsize=(10, 6))
sns.boxplot(x='hotel', y='adr', hue='is_canceled', data=df, palette='Pastel1', showfliers=False)
plt.title('ADR Distribution by Hotel Type and Cancellation Status', fontsize=16)
plt.xlabel('Hotel Type', fontsize=12)
plt.ylabel('Average Daily Rate (ADR)', fontsize=12)
plt.legend(title='Canceled', labels=['No (0)', 'Yes (1)'])
plt.tight_layout()
plt.show()
plt.savefig(os.path.join(output_dir, 'adr_by_hotel_and_cancellation.png'))
plt.close()


## 3. Export Cleaned Data

The final cleaned dataset is exported to a CSV file, ready for import into Microsoft Excel for dashboard creation.


In [ ]:

output_path = '/home/ubuntu/hotel_bookings_Cleaned.csv'
df.to_csv(output_path, index=False)
print(f"Cleaned data successfully exported to: {output_path}")
